# Auto-Merging Retreival - Source Group B

In [30]:
import os

from llama_index.core import (
    VectorStoreIndex,
    StorageContext,
    Settings,
    load_index_from_storage,
)
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import HierarchicalNodeParser, get_leaf_nodes
from llama_index.core.retrievers import AutoMergingRetriever
from llama_index.core.indices.postprocessor import SentenceTransformerRerank
from llama_index.core.query_engine import RetrieverQueryEngine


def build_automerging_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index",
    chunk_sizes=None,
):
    # Set global settings
    Settings.llm = llm
    Settings.embed_model = embed_model

    chunk_sizes = chunk_sizes or [2048, 512, 128]
    node_parser = HierarchicalNodeParser.from_defaults(chunk_sizes=chunk_sizes)
    nodes = node_parser.get_nodes_from_documents(documents)
    leaf_nodes = get_leaf_nodes(nodes)

    storage_context = StorageContext.from_defaults()
    storage_context.docstore.add_documents(nodes)

    if not os.path.exists(save_dir):
        automerging_index = VectorStoreIndex(
            leaf_nodes,
            storage_context=storage_context,
        )
        automerging_index.storage_context.persist(persist_dir=save_dir)
    else:
        automerging_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=save_dir),
        )
    return automerging_index


def get_automerging_query_engine(
    automerging_index,
    similarity_top_k=12,
    rerank_top_n=6,
):
    base_retriever = automerging_index.as_retriever(similarity_top_k=similarity_top_k)
    retriever = AutoMergingRetriever(
        base_retriever, automerging_index.storage_context, verbose=True
    )
    rerank = SentenceTransformerRerank(
        top_n=rerank_top_n, model="BAAI/bge-reranker-base"
    )
    auto_merging_engine = RetrieverQueryEngine.from_args(
        retriever, node_postprocessors=[rerank]
    )
    return auto_merging_engine


In [31]:
from llama_index.llms.openai import OpenAI 
from llama_index.core import Settings

from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["./SourceGroupB/source12.pdf", "./SourceGroupB/source1.pdf", "./SourceGroupB/source2.pdf", "./SourceGroupB/source3.pdf", "./SourceGroupB/source4.pdf", "./SourceGroupB/source5.pdf", "./SourceGroupB/source6.pdf", "./SourceGroupB/source7.pdf", "./SourceGroupB/source8.pdf", "./SourceGroupB/source9.pdf", "./SourceGroupB/source10.pdf", "./SourceGroupB/source11.pdf"]
).load_data()


In [32]:
from llama_index.core import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

index = build_automerging_index(
    [document],
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.5),
    save_dir="./merging_index_GroupB",
)


In [33]:
from llama_index.core.response.notebook_utils import display_response

query_engine = get_automerging_query_engine(index, similarity_top_k=6)

with open('./generated_questions.txt') as file:
    for line in file:
        question = line.strip()
        print(question)
        display_response(query_engine.query(question))

Are current policing methods effective in reducing crime in minority communities?


**`Final Response:`** Current policing methods are not effective in reducing crime in minority communities.

How does the use of facial recognition technology impact racial profiling?


**`Final Response:`** The use of facial recognition technology can lead to racial profiling as these systems have been shown to disproportionately fail in detecting Black faces while still targeting Black individuals for surveillance. This results in a situation where Black individuals are more likely to be unfairly monitored and scrutinized due to the biases and inaccuracies present in facial recognition technology.

Is there evidence to support claims of systemic racism within law enforcement agencies?
> Merging 1 nodes into parent node.
> Parent node id: 37fd3e8d-3f89-4e6c-b2fb-1a9ab9cdb6c2.
> Parent node text: 5/4/25, 8:12 PM Systemic racism pervades US police and justice systems, UN Mechanism on Racial Ju...



...

**`Final Response:`** Underreporting hate crimes on minority communities can lead to a lack of awareness and acknowledgment of the extent of discrimination and violence they face. This can result in inadequate support and resources being allocated to address these issues, perpetuating a cycle of marginalization and injustice. Additionally, underreporting can contribute to a false sense of security and downplay the severity of the challenges that minority communities encounter, hindering efforts to combat hate crimes effectively.

How do law enforcement agencies address allegations of racial bias in their practices?
> Merging 3 nodes into parent node.
> Parent node id: 832eca2c-1aed-492a-bcde-8523e0a5a08e.
> Parent node text: By  centering  marginalized  perspectives,  we  move  beyond  fixing  algorithmic  bias  to  
fun...



**`Final Response:`** Law enforcement agencies address allegations of racial bias in their practices by putting in place alternative responses to policing, addressing systemic racism against Black law enforcement officers, and tackling issues of white supremacy ideology within their agencies.

What measures can be taken to ensure equitable treatment of all racial groups within the criminal justice system?
> Merging 4 nodes into parent node.
> Parent node id: 832eca2c-1aed-492a-bcde-8523e0a5a08e.
> Parent node text: By  centering  marginalized  perspectives,  we  move  beyond  fixing  algorithmic  bias  to  
fun...



**`Final Response:`** Measures such as implementing alternative responses to policing, addressing systemic racism within police agencies, promoting a culture of respect and wellbeing within law enforcement ranks, and combating prevailing impunity through joint efforts of all actors involved, including police departments and unions, can help ensure equitable treatment of all racial groups within the criminal justice system.